# Example Method

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import re
from sklearn.model_selection import train_test_split

/home/cer/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
# max_data_size = 320000

In [4]:
train_df = pd.read_csv('../input/en_train.csv')
test_df = pd.read_csv('../input/en_test.csv')
sample_df = pd.read_csv('../input/en_sample_submission.csv')

In [7]:
y_data[:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0])

In [9]:
labels = train_df["class"].unique()
labels

array(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM',
       'DECIMAL', 'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC',
       'DIGIT', 'FRACTION', 'TELEPHONE', 'ADDRESS'], dtype=object)

In [11]:
class2index = dict(zip(labels, range(len(labels))))
class2index

{'ADDRESS': 15,
 'CARDINAL': 4,
 'DATE': 2,
 'DECIMAL': 6,
 'DIGIT': 12,
 'ELECTRONIC': 11,
 'FRACTION': 13,
 'LETTERS': 3,
 'MEASURE': 7,
 'MONEY': 8,
 'ORDINAL': 9,
 'PLAIN': 0,
 'PUNCT': 1,
 'TELEPHONE': 14,
 'TIME': 10,
 'VERBATIM': 5}

In [18]:
for k in class2index:
    if k in ["PLAIN", "PUNCT"]:
        class2index[k] = 0
    else:
        class2index[k] = 1
class2index

{'ADDRESS': 1,
 'CARDINAL': 1,
 'DATE': 1,
 'DECIMAL': 1,
 'DIGIT': 1,
 'ELECTRONIC': 1,
 'FRACTION': 1,
 'LETTERS': 1,
 'MEASURE': 1,
 'MONEY': 1,
 'ORDINAL': 1,
 'PLAIN': 0,
 'PUNCT': 0,
 'TELEPHONE': 1,
 'TIME': 1,
 'VERBATIM': 1}

In [19]:
y_data = np.array(map(lambda x: class2index[x], train_df['class'].values))

In [16]:
y_data[:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0])

In [ ]:
y_data =  pd.factorize(train_df['class'])

In [5]:
x_data = []
# 将类别数字化
y_data =  pd.factorize(train_df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()
# 每个目标词用组成这个词的所有字符的ascii码表示，并padding
for x in train_df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

In [5]:
x_data[:10]

[array([ 66, 114, 105, 108, 108,  97, 110, 116,  97, 105]),
 array([105, 115,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([97,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([103, 101, 110, 117, 115,   0,   0,   0,   0,   0]),
 array([111, 102,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([112, 108,  97, 110, 116,   0,   0,   0,   0,   0]),
 array([105, 110,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([102,  97, 109, 105, 108, 121,   0,   0,   0,   0]),
 array([ 65,  99,  97, 110, 116, 104,  97,  99, 101,  97]),
 array([46,  0,  0,  0,  0,  0,  0,  0,  0,  0])]

In [6]:
del train_df
gc.collect()

7

In [20]:
def context_window_transform(data, pad_size):
    """每个词加上前面一个和后面一个词，中间用-1隔开"""
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row += [boundary_letter]
            row += x.tolist()
        row += [boundary_letter]
        neo_data.append(row)
    return neo_data

In [21]:
# x_data = x_data[:max_data_size]
# y_data = y_data[:max_data_size]
x_data_context = np.array(context_window_transform(x_data, pad_size))

In [19]:
x_data_context[:10]

array([[  -1.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,   -1.,   66.,  114.,  105.,  108.,  108.,   97.,
         110.,  116.,   97.,  105.,   -1.,  105.,  115.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,   -1.],
       [  -1.,   66.,  114.,  105.,  108.,  108.,   97.,  110.,  116.,
          97.,  105.,   -1.,  105.,  115.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,   -1.,   97.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,   -1.],
       [  -1.,  105.,  115.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,   -1.,   97.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,   -1.,  103.,  101.,  110.,  117.,
         115.,    0.,    0.,    0.,    0.,    0.,   -1.],
       [  -1.,   97.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,   -1.,  103.,  101.,  110.,  117.,  115.,    0.,
           0.,    0.,    0.,    0.,   -1.,  1

In [22]:
del x_data
gc.collect()

0

In [23]:
x_data_context_a = np.array(x_data_context)
y_data_a = np.array(y_data)

print('Total number of samples:', len(x_data_context))

print('x_data sample:')
print(x_data_context[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

('Total number of samples:', 9918441)
x_data sample:
[  -1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -1.
   66.  114.  105.  108.  108.   97.  110.  116.   97.  105.   -1.  105.
  115.    0.    0.    0.    0.    0.    0.    0.    0.   -1.]
y_data sample:
0
labels:
Index([u'PLAIN', u'PUNCT', u'DATE', u'LETTERS', u'CARDINAL', u'VERBATIM',
       u'DECIMAL', u'MEASURE', u'MONEY', u'ORDINAL', u'TIME', u'ELECTRONIC',
       u'DIGIT', u'FRACTION', u'TELEPHONE', u'ADDRESS'],
      dtype='object')


In [25]:
del x_data_context
del y_data
gc.collect()

0

In [26]:
x_train, x_valid, y_train, y_valid= train_test_split(x_data_context_a, y_data_a,
                                                      test_size=0.1, random_state=2017)

In [27]:
del x_data_context_a
del y_data_a
gc.collect()

0

In [28]:
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

In [29]:
del x_train
del y_train
gc.collect()

14

In [ ]:
param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)

[0]	valid-merror:0.009588	train-merror:0.009344
Multiple eval metrics have been passed: 'train-merror' will be used for early stopping.

Will train until train-merror hasn't improved in 20 rounds.


In [ ]:
pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

In [ ]:
gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

model.save_model(os.path.join(out_path, 'xgb_model'))